# Human V Machine Text Classifier

**About:** In this project, we trained a feedforward neural network to distinguish between human-written and machine-genrated text by using a hybrid approach that integrated semantic and stylic features of the input text. This approach is similar to the techniques discussed in [\[1\]](https://openreview.net/pdf?id=cWiEN1plhJ) and [\[2\]](https://arxiv.org/pdf/2505.14608). The semantic features are gotten from embeddings generated using a [BERT-base-uncased](https://huggingface.co/google-bert/bert-base-uncased) model that we further finetuned for the task of human-machine text classification using the [Human Vs. LLM Text Corpus](https://www.kaggle.com/datasets/starblasters8/human-vs-llm-text-corpus?resource=download) dataset. The style features were extracted as embeddings in our data preprocessing notebook using the [StyleDistance](https://huggingface.co/StyleDistance/styledistance) model that was trained to separate semantics from style for applications like authorship detection. After concatenating the embeddings from the finetuned BERT model and those from StyleDistance, we used this as input for training a zero-shot 3-layer feed-forward neural network (or muli-layer perceptron) that can delienate human and machine text with a >96% accuracy.

In [14]:
# !pip install transformers datasets evaluate

## Import and Inspect Data

**Description:** Loading and preloading the training, testing, and validation datasets prepared in our data preprocessing step.

In [1]:
# Uncomment to map drive folder in Google Colab if needed
# from google.colab import drive
# drive.mount('/content/drive')

In [3]:
import pandas as pd

data_dir = '/content/drive/MyDrive/human_v_machine_data'
train_df = pd.read_parquet(f'{data_dir}/train_data.parquet') #.head(1000)
test_df = pd.read_parquet(f'{data_dir}/test_data.parquet') #.head(200)
val_df = pd.read_parquet(f'{data_dir}/val_data.parquet') #.head(200)

**Training dataset**

In [4]:
# Inspect training set
row_count, col_count = train_df.shape
print(f'Number of rows: {row_count}')
print(f'Number of columns: {col_count}')
train_df.head()

Number of rows: 403752
Number of columns: 7


,text,source,prompt_id,text_length,word_count,labels,style_embedding
0,In the ever-evolving world of agriculture and...,Neural-Chat-7B,1481,3579,492,1,"[-0.5739298, 0.53654164, 0.48202765, 0.2068162..."
1,Scotland's Education Secretary John Swinney ha...,GPT-3.5,0,1835,288,1,"[-0.7382925, 0.54586464, 0.4748918, 0.25404334..."
2,Old age caught up on Jimmy Santry like an Olym...,Human,0,885,155,0,"[-0.49520755, 0.5784473, 0.5190531, -0.0773673..."
3,It was a typical weekday evening and the famil...,GPT-3.5,0,548,96,1,"[-0.6054464, 0.47413713, 0.37590465, 0.2668135..."
4,Waymo just announced that it hit 10 million mi...,Bloom-7B,0,1807,276,1,"[-0.4366107, 0.14334065, 0.46985778, 0.4575423..."


**Test dataset**

In [5]:
# Inspect test set
row_count, col_count = test_df.shape
print(f'Number of rows: {row_count}')
print(f'Number of columns: {col_count}')
test_df.head()

Number of rows: 50469
Number of columns: 7


,text,source,prompt_id,text_length,word_count,labels,style_embedding
0,Janice and her family were going on a ride. Th...,T0-11B,0,180,36,1,"[-0.50312734, 0.49528196, 0.36710575, 0.287121..."
1,"Well, first of all, there's tons. The most fam...",Text-Davinci-003,0,979,153,1,"[-0.6014539, 0.38711172, 0.43624958, 0.1534785..."
2,The area's many colleges and universities make...,Human,0,650,96,0,"[-0.7674345, 0.4485233, 0.4587153, 0.2998681, ..."
3,"Uber drivers dissatisfied with shrinking pay, ...",Human,0,593,99,0,"[-0.7241476, 0.491437, 0.3187004, 0.2684327, -..."
4,Title: The Vital Role of Education in Fosterin...,StripedHyena-Nous-7B,1249,3616,473,1,"[-0.5393813, 0.5707216, 0.5645622, 0.24652848,..."


**Validation dataset**

In [6]:
# Inspect validation set
row_count, col_count = val_df.shape
print(f'Number of rows: {row_count}')
print(f'Number of columns: {col_count}')
val_df.head()

Number of rows: 50469
Number of columns: 7


,text,source,prompt_id,text_length,word_count,labels,style_embedding
0,European Union law is applied by the courts of...,Flan-T5-Base,0,282,50,1,"[0.14726105, 0.2817755, 0.39384592, -0.0151012..."
1,"Part I: ""Send a message to Fort rfunny,"" spoke...",Human,0,1846,328,0,"[-0.27435556, 0.41265067, 0.4729457, 0.1368453..."
2,Sarah's cat Dusty was a very picky eater. She ...,OPT-30B,0,347,67,1,"[-0.4449138, 0.040379003, 0.18393947, -0.04828..."
3,"Well, sometimes when there are a lot of people...",GPT-3.5,0,499,97,1,"[-0.385399, 0.3046976, 0.26150247, 0.36810344,..."
4,shoots areas all around the park while the bag...,GPT-J,0,1754,284,1,"[-0.63270855, 0.49869356, 0.47112256, 0.185904..."


## Extract Data for Finetuning BERT-base-uncased Model

**Description:** Get the `text` and `labels` columns from the datasets.


In [7]:
zsl_train_df = train_df[['text', 'labels']]
zsl_test_df = test_df[['text', 'labels']]
zsl_val_df = val_df[['text', 'labels']]

print("First few rows of zsl_train_df:")
print(zsl_train_df.head())

First few rows of zsl_train_df:
                                                text  labels
0   In the ever-evolving world of agriculture and...       1
1  Scotland's Education Secretary John Swinney ha...       1
2  Old age caught up on Jimmy Santry like an Olym...       0
3  It was a typical weekday evening and the famil...       1
4  Waymo just announced that it hit 10 million mi...       1


## Preprocess Text Data for Fine-tuning

**Description:** Tokenize the text from the `text` column, to be used for finetuning the `bert-base-uncased` model for the downstream task of human-machine text classification.


In [8]:
from transformers import AutoTokenizer

# Load a pre-trained tokenizer
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')

print("Tokenizer loaded successfully.")

Tokenizer loaded successfully.


In [9]:
def tokenize_function(examples):
    return tokenizer(examples['text'], padding='max_length', truncation=True, max_length=512)

print("Tokenization function defined.")

Tokenization function defined.


In [10]:
from datasets import Dataset

# Convert DataFrames to Dataset objects
tokenized_train_dataset = Dataset.from_pandas(zsl_train_df)
tokenized_test_dataset = Dataset.from_pandas(zsl_test_df)
tokenized_val_dataset = Dataset.from_pandas(zsl_val_df)

# Apply the tokenization function to each dataset
tokenized_train_dataset = tokenized_train_dataset.map(tokenize_function, batched=True)
tokenized_test_dataset = tokenized_test_dataset.map(tokenize_function, batched=True)
tokenized_val_dataset = tokenized_val_dataset.map(tokenize_function, batched=True)

print("First element of tokenized_train_dataset:")
print(tokenized_train_dataset[0])

Map:   0%|          | 0/403752 [00:00<?, ? examples/s]

Map:   0%|          | 0/50469 [00:00<?, ? examples/s]

Map:   0%|          | 0/50469 [00:00<?, ? examples/s]

First element of tokenized_train_dataset:
{'text': ' In the ever-evolving world of agriculture and food production, it is essential to acknowledge the impact of biotechnology on sustainable food systems. This innovative approach to food production has the potential to revolutionize our current ways of growing and harvesting crops, while simultaneously addressing global concerns surrounding climate change, food scarcity, and environmental degradation. The role of biotechnology in sustainable food systems is multifaceted, encompassing various aspects such as genetic engineering, molecular biology, and bioinformatics.\n\nGenetic Engineering: One of the most prominent applications of biotechnology in sustainable food systems is through genetic engineering. By manipulating the genetic makeup of plants, scientists can enhance their tolerance to environmental stresses, improve their nutritional value, and boost their overall productivity. This has led to a range of genetically modified crops 

## Load Pre-trained Foundation Model `(bert-base-uncased)`

**Description:** Load `bert-base-uncased` model with `num_labels=2` to prepare it for binary sequence classification


In [11]:
from transformers import AutoModelForSequenceClassification

# Load a pre-trained Transformer model for sequence classification
model = AutoModelForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)

print("Pre-trained model for sequence classification loaded successfully.")

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Pre-trained model for sequence classification loaded successfully.


_*Also drop text column, since the text has been tokenized and the raw text is not needed for finetuning._


In [12]:
tokenized_train_dataset = tokenized_train_dataset.remove_columns(["text"])
tokenized_test_dataset = tokenized_test_dataset.remove_columns(["text"])
tokenized_val_dataset = tokenized_val_dataset.remove_columns(["text"])

print("Unnecessary columns removed in all datasets.")
print("First element of tokenized_train_dataset after column removal:")
print(tokenized_train_dataset[0])

Unnecessary columns removed in all datasets.
First element of tokenized_train_dataset after column removal:
{'labels': 1, 'input_ids': [101, 1999, 1996, 2412, 1011, 20607, 2088, 1997, 5237, 1998, 2833, 2537, 1010, 2009, 2003, 6827, 2000, 13399, 1996, 4254, 1997, 20353, 2006, 9084, 2833, 3001, 1012, 2023, 9525, 3921, 2000, 2833, 2537, 2038, 1996, 4022, 2000, 4329, 4697, 2256, 2783, 3971, 1997, 3652, 1998, 21534, 8765, 1010, 2096, 7453, 12786, 3795, 5936, 4193, 4785, 2689, 1010, 2833, 11228, 12972, 1010, 1998, 4483, 16627, 1012, 1996, 2535, 1997, 20353, 1999, 9084, 2833, 3001, 2003, 4800, 12172, 3064, 1010, 19129, 2536, 5919, 2107, 2004, 7403, 3330, 1010, 8382, 7366, 1010, 1998, 16012, 2378, 14192, 17592, 1012, 7403, 3330, 1024, 2028, 1997, 1996, 2087, 4069, 5097, 1997, 20353, 1999, 9084, 2833, 3001, 2003, 2083, 7403, 3330, 1012, 2011, 26242, 1996, 7403, 5789, 1997, 4264, 1010, 6529, 2064, 11598, 2037, 13986, 2000, 4483, 23253, 1010, 5335, 2037, 28268, 3643, 1010, 1998, 12992, 2037, 3452

## Define Custom evaluation function

**Description:** We define a custom function to specify accuracy as the metric that should be optimized during model finetuning.

In [13]:
import numpy as np
from evaluate import load

def compute_metrics(eval_pred):
    metric = load("accuracy")
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return metric.compute(predictions=predictions, references=labels)

print("compute_metrics function defined with corrected import.")

compute_metrics function defined with corrected import.


## Initialize and Configure Data Collator, Training Arguments, and Trainer

**Description:** Initialize and configure the data collator, the training arguments, and the trainer objects that will be used for finetuning the `bert-base-uncased` model.

In [14]:
from transformers import DataCollatorWithPadding, TrainingArguments, Trainer

# Initialize Data Collator
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

# Configure TrainingArguments
training_args = TrainingArguments(
    output_dir='/content/drive/MyDrive/human_v_machine_results',
    eval_strategy='epoch',
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=500,
    save_strategy='epoch',
    load_best_model_at_end=True,
    metric_for_best_model='accuracy',
    push_to_hub=False,
    report_to="none"
)

# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_val_dataset,
    processing_class=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

print("Data Collator, Training Arguments, and Trainer initialized successfully.")

Data Collator, Training Arguments, and Trainer initialized successfully.


## Fine-tune Foundation Model for Zeroshot Learning (ZSL)

**Description:** Kickoff the training process to finetune the foundation model, i.e. `bert-base-uncased`, for the downstream task of binary classification for human-machine text.

In [15]:
trainer.train()
print("Fine-tuning initiated.")

Epoch,Training Loss,Validation Loss,Accuracy
1,0.154700,0.232364,0.955914


Epoch,Training Loss,Validation Loss,Accuracy
1,0.154700,0.232364,0.955914
2,0.070200,0.260700,0.953714
3,0.027000,0.314154,0.954051


Fine-tuning initiated.


## Extract Fine-tuned BERT Embeddings

**Description:** Used the task-specific/finetuned BERT model to generate semantic embeddings that will be used with style embeddings to train a lighter weight neural network for the task of classification. Based on the evaluation scores shown above (i.e. training loss, validation loss, and accuracy), the finetuned model after the first epoch was used to generating the embeddings. The decreasing trainig loss and increasing validation loss show that the model started overfitting after the first epoch. It also helps that epoch 1 had the most accurate model, although roughly similar to the others.


In [16]:
import torch
import numpy as np
from tqdm.auto import tqdm

from transformers import AutoTokenizer, AutoModelForSequenceClassification

# Load the fine-tuned model (assuming it was saved or is still in memory from the previous step)
# If the model needs to be reloaded from a saved checkpoint:
model_path = "/content/drive/MyDrive/human_v_machine_results/checkpoint-50469" # Replace with actual checkpoint path if needed
model = AutoModelForSequenceClassification.from_pretrained(model_path)

# Load the tokenizer again if it's not already in scope
# tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')

# Set up device for computation
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Move model to device and set to evaluation mode
model.to(device)
model.eval()

print("Model moved to device and set to evaluation mode.")

# Define the function to extract BERT embeddings
def get_bert_embeddings(texts, model, tokenizer, device, max_length=512, batch_size=32):
    all_embeddings = []
    # Process texts in batches
    for i in tqdm(range(0, len(texts), batch_size), desc="Extracting embeddings"):
        batch_texts = texts[i:i+batch_size]

        # Tokenize the batch
        inputs = tokenizer(batch_texts, return_tensors='pt', padding=True, truncation=True, max_length=max_length)
        inputs = {k: v.to(device) for k, v in inputs.items()}

        with torch.no_grad():
            # Access the base BERT model to get hidden states
            # model.bert is the base BERT model within AutoModelForSequenceClassification
            outputs = model.bert(**inputs, output_hidden_states=True)
            # Extract [CLS] token embedding from the last hidden layer
            cls_embeddings = outputs.last_hidden_state[:, 0, :].cpu().numpy()
            all_embeddings.append(cls_embeddings)

    return np.vstack(all_embeddings)

# Apply the function to extract embeddings for each DataFrame
train_df['bert_embedding'] = get_bert_embeddings(train_df['text'].tolist(), model, tokenizer, device).tolist()
test_df['bert_embedding'] = get_bert_embeddings(test_df['text'].tolist(), model, tokenizer, device).tolist()
val_df['bert_embedding'] = get_bert_embeddings(val_df['text'].tolist(), model, tokenizer, device).tolist()

print("BERT embeddings extracted successfully for all datasets.")

# Display the first few rows of train_df with 'text' and 'bert_embedding'
print("\nFirst few rows of train_df with BERT embeddings:")
print(train_df[['text', 'bert_embedding']].head())

Using device: cuda
Model moved to device and set to evaluation mode.


Extracting embeddings:   0%|          | 0/12618 [00:00<?, ?it/s]

Extracting embeddings:   0%|          | 0/1578 [00:00<?, ?it/s]

Extracting embeddings:   0%|          | 0/1578 [00:00<?, ?it/s]

BERT embeddings extracted successfully for all datasets.

First few rows of train_df with BERT embeddings:
                                                text  \
0   In the ever-evolving world of agriculture and...   
1  Scotland's Education Secretary John Swinney ha...   
2  Old age caught up on Jimmy Santry like an Olym...   
3  It was a typical weekday evening and the famil...   
4  Waymo just announced that it hit 10 million mi...   

                                      bert_embedding  
0  [-0.33051419258117676, 2.0117979049682617, 0.1...  
1  [-0.3298913836479187, 2.015545606613159, 0.156...  
2  [0.3176843225955963, -1.0377979278564453, 0.00...  
3  [-0.3322458863258362, 2.021146535873413, 0.154...  
4  [-0.34231674671173096, 2.0080692768096924, 0.1...  


## Concatenate Semantic and Stylistic Features

**Description:** Concatenate the fine-tuned BERT embeddings and style embeddings into a single `integrated_embedding` column for `train_df`, `test_df`, and `val_df`.


In [17]:
import numpy as np

def integrate_embeddings(df):
    # Ensure 'bert_embedding' and 'style_embedding' are lists of arrays/lists
    bert_embeddings = np.array(df['bert_embedding'].tolist())
    style_embeddings = np.array(df['style_embedding'].tolist())

    # Concatenate horizontally (axis=1)
    df['integrated_embedding'] = [np.concatenate((b, s), axis=0).tolist()
                                  for b, s in zip(bert_embeddings, style_embeddings)]
    return df

# Apply the function to each DataFrame
train_df = integrate_embeddings(train_df)
test_df = integrate_embeddings(test_df)
val_df = integrate_embeddings(val_df)

print("Embeddings integrated successfully for all dataframes.")

# Display the first few rows of train_df with 'integrated_embedding'
print("\nFirst few rows of train_df with integrated_embedding:")
print(train_df[['text', 'integrated_embedding']].head())

# Print the shape of the first 'integrated_embedding' entry in train_df
if not train_df.empty:
    first_embedding_shape = np.array(train_df['integrated_embedding'].iloc[0]).shape
    print(f"\nShape of the first integrated_embedding in train_df: {first_embedding_shape}")

Embeddings integrated successfully for all dataframes.

First few rows of train_df with integrated_embedding:
                                                text  \
0   In the ever-evolving world of agriculture and...   
1  Scotland's Education Secretary John Swinney ha...   
2  Old age caught up on Jimmy Santry like an Olym...   
3  It was a typical weekday evening and the famil...   
4  Waymo just announced that it hit 10 million mi...   

                                integrated_embedding  
0  [-0.33051419258117676, 2.0117979049682617, 0.1...  
1  [-0.3298913836479187, 2.015545606613159, 0.156...  
2  [0.3176843225955963, -1.0377979278564453, 0.00...  
3  [-0.3322458863258362, 2.021146535873413, 0.154...  
4  [-0.34231674671173096, 2.0080692768096924, 0.1...  

Shape of the first integrated_embedding in train_df: (1536,)


# Save updated dataset containing integrated_embedding column

**Description:** Save the current state of the datasets for easier/quicker access to the embeddings for possible future use. This will save time and compute resources for regenerating the embeddings if needed.

In [19]:
import os

# 1. Define a directory path where the processed Parquet files will be saved
processed_data_dir = '/content/drive/MyDrive/human_v_machine_data_processed'
os.makedirs(processed_data_dir, exist_ok=True)

# 2. Save the train_df DataFrame
train_df.to_parquet(f'{processed_data_dir}/train_data_integrated.parquet', index=False)

# 3. Save the test_df DataFrame
test_df.to_parquet(f'{processed_data_dir}/test_data_integrated.parquet', index=False)

# 4. Save the val_df DataFrame
val_df.to_parquet(f'{processed_data_dir}/val_data_integrated.parquet', index=False)

# 5. Print a confirmation message
print(f"All DataFrames with integrated embeddings saved successfully to {processed_data_dir}.")

All DataFrames with integrated embeddings saved successfully to /content/drive/MyDrive/human_v_machine_data_processed.


# Task
Define the architecture of the integrated deep learning classification model that will use the fine-tuned BERT foundation and accept concatenated feature vectors, serving as the base for both ZSL and FSL tasks.

## Define Neural Network Model Architecture

**Description:** Define the architectrue of the neural network model for zeroshot learning on the task of human-machine text classification. This is a 3-layer multi-layer perceptron that will build upon the fine-tuned BERT by accepting the concatenated style and semantic feature vectors of a given text and predict whether the underlying text was human-written or machine-generated.


In [20]:
import torch
import torch.nn as nn

class IntegratedClassifier(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(IntegratedClassifier, self).__init__()
        self.classifier = nn.Sequential(
            nn.Linear(input_dim, 512), # First linear layer
            nn.ReLU(),                 # Activation function
            nn.Dropout(0.3),           # Dropout for regularization
            nn.Linear(512, 256),       # Second linear layer
            nn.ReLU(),                 # Activation function
            nn.Dropout(0.3),           # Dropout for regularization
            nn.Linear(256, output_dim) # Output layer
        )

    def forward(self, x):
        return self.classifier(x)

print("IntegratedClassifier model architecture defined.")

IntegratedClassifier model architecture defined.


## Train the Human-Machine Text Classification Model

**Description:** Run the training process for the model architecture defined above. The code below will save both the best model encountered across all epochs and the final model at the last epoch.


In [21]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from tqdm.auto import tqdm
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import numpy as np
import copy # Import copy module for deep copying model state

# 2. Determine if a GPU is available and set the device variable accordingly.
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# 3. Define a custom PyTorch Dataset class
class IntegratedEmbeddingsDataset(Dataset):
    def __init__(self, embeddings, labels):
        self.embeddings = torch.tensor(embeddings, dtype=torch.float32)
        self.labels = torch.tensor(labels, dtype=torch.long)

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return self.embeddings[idx], self.labels[idx]

# 4. Prepare the training and validation data
# Convert 'integrated_embedding' to numpy arrays, handle potential nested lists
train_embeddings = np.array([np.array(e) for e in train_df['integrated_embedding'].tolist()])
val_embeddings = np.array([np.array(e) for e in val_df['integrated_embedding'].tolist()])

train_labels = train_df['labels'].to_numpy()
val_labels = val_df['labels'].to_numpy()

train_dataset = IntegratedEmbeddingsDataset(train_embeddings, train_labels)
val_dataset = IntegratedEmbeddingsDataset(val_embeddings, val_labels)

# 5. Create PyTorch DataLoader instances
batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

print("Datasets and DataLoaders created successfully.")

# 6. Determine input_dim and output_dim, and instantiate the IntegratedClassifier model
input_dim = train_embeddings.shape[1] # Dimension of the integrated embeddings
output_dim = 2 # Binary classification (human vs. machine)

# The IntegratedClassifier class definition is expected to be available from a previous cell
model = IntegratedClassifier(input_dim, output_dim).to(device)
print(f"IntegratedClassifier instantiated with input_dim={input_dim}, output_dim={output_dim} and moved to {device}.")

# 7. Define the loss function and the optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
print("Loss function and optimizer defined.")

# 8. Implement the training loop
num_epochs = 10
print(f"Starting training for {num_epochs} epochs...")

# Initialize variables for best model saving
best_val_accuracy = -1.0
best_model_state_dict = None

for epoch in range(num_epochs):
    # Training phase
    model.train()
    total_train_loss = 0
    for batch_embeddings, batch_labels in tqdm(train_loader, desc=f"Training Epoch {epoch+1}"):
        batch_embeddings, batch_labels = batch_embeddings.to(device), batch_labels.to(device)

        optimizer.zero_grad()
        outputs = model(batch_embeddings)
        loss = criterion(outputs, batch_labels)
        loss.backward()
        optimizer.step()
        total_train_loss += loss.item()

    avg_train_loss = total_train_loss / len(train_loader)

    # Evaluation phase
    model.eval()
    total_val_loss = 0
    all_preds = []
    all_labels = []
    with torch.no_grad():
        for batch_embeddings, batch_labels in tqdm(val_loader, desc=f"Validation Epoch {epoch+1}"):
            batch_embeddings, batch_labels = batch_embeddings.to(device), batch_labels.to(device)

            outputs = model(batch_embeddings)
            loss = criterion(outputs, batch_labels)
            total_val_loss += loss.item()

            _, predicted = torch.max(outputs.data, 1)
            all_preds.extend(predicted.cpu().numpy())
            all_labels.extend(batch_labels.cpu().numpy())

    avg_val_loss = total_val_loss / len(val_loader)

    accuracy = accuracy_score(all_labels, all_preds)
    precision, recall, f1, _ = precision_recall_fscore_support(all_labels, all_preds, average='binary', zero_division=0)

    print(f"Epoch {epoch+1}/{num_epochs} - Train Loss: {avg_train_loss:.4f}, Val Loss: {avg_val_loss:.4f}, Accuracy: {accuracy:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}, F1: {f1:.4f}")

    # Save the best model based on validation accuracy
    if accuracy > best_val_accuracy:
        best_val_accuracy = accuracy
        best_model_state_dict = copy.deepcopy(model.state_dict())
        print(f"New best model saved with validation accuracy: {best_val_accuracy:.4f}")

print("Training complete.")

# 9. Save the best trained model's state dictionary after the loop completes
if best_model_state_dict is not None:
    model_save_path_best = '/content/drive/MyDrive/human_v_machine_models/integrated_classifier_zsl_best.pth'
    torch.save(best_model_state_dict, model_save_path_best)
    print(f"Best ZSL model saved to {model_save_path_best} with accuracy {best_val_accuracy:.4f}")
else:
    print("No model was saved as no best_model_state_dict was found.")

Using device: cuda
Datasets and DataLoaders created successfully.
IntegratedClassifier instantiated with input_dim=1536, output_dim=2 and moved to cuda.
Loss function and optimizer defined.
Starting training for 10 epochs...


Training Epoch 1:   0%|          | 0/12618 [00:00<?, ?it/s]

Validation Epoch 1:   0%|          | 0/1578 [00:00<?, ?it/s]

Epoch 1/10 - Train Loss: 0.0510, Val Loss: 0.0888, Accuracy: 0.9671, Precision: 0.9740, Recall: 0.9725, F1: 0.9732
New best model saved with validation accuracy: 0.9671


Training Epoch 2:   0%|          | 0/12618 [00:00<?, ?it/s]

...

Validation Epoch 9:   0%|          | 0/1578 [00:00<?, ?it/s]

Epoch 9/10 - Train Loss: 0.0451, Val Loss: 0.0944, Accuracy: 0.9667, Precision: 0.9791, Recall: 0.9665, F1: 0.9728


Training Epoch 10:   0%|          | 0/12618 [00:00<?, ?it/s]

Validation Epoch 10:   0%|          | 0/1578 [00:00<?, ?it/s]

Epoch 10/10 - Train Loss: 0.0450, Val Loss: 0.0868, Accuracy: 0.9673, Precision: 0.9783, Recall: 0.9683, F1: 0.9733
Training complete.
Best ZSL model saved to /content/drive/MyDrive/human_v_machine_models/integrated_classifier_zsl_best.pth with accuracy 0.9679


## Evaluate the Trained Model's Performance

**Description:** Evaluate the performance of the trained model on unseen human and LLM outputs using appropriate metrics such as accuracy, precision, recall, and F1-score. This evaluation uses the test dataset, which is completely separate from the training and validation datasets used for training the model.


In [22]:
import torch
import numpy as np
from torch.utils.data import DataLoader
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from tqdm.auto import tqdm

# 1. Prepare the test_df data for evaluation
# Convert 'integrated_embedding' to numpy arrays, handle potential nested lists
test_embeddings = np.array([np.array(e) for e in test_df['integrated_embedding'].tolist()])
test_labels = test_df['labels'].to_numpy()

# 2. Create an instance of the IntegratedEmbeddingsDataset
test_dataset = IntegratedEmbeddingsDataset(test_embeddings, test_labels)

# 3. Create a PyTorch DataLoader for the test dataset
batch_size = 32
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

print("Test Dataset and DataLoader created successfully.")

# 4. Ensure the IntegratedClassifier model is moved to the appropriate device and set to evaluation mode
model.to(device)
model.eval()
print(f"Model moved to {device} and set to evaluation mode.")

# 5. Implement an evaluation loop for the test set
total_test_loss = 0
all_test_preds = []
all_test_labels = []

print("Starting evaluation on the test set...")

with torch.no_grad():
    for batch_embeddings, batch_labels in tqdm(test_loader, desc="Evaluating Test Set"):
        batch_embeddings, batch_labels = batch_embeddings.to(device), batch_labels.to(device)

        outputs = model(batch_embeddings)
        loss = criterion(outputs, batch_labels)
        total_test_loss += loss.item()

        _, predicted = torch.max(outputs.data, 1)
        all_test_preds.extend(predicted.cpu().numpy())
        all_test_labels.extend(batch_labels.cpu().numpy())

avg_test_loss = total_test_loss / len(test_loader)

# 6. Calculate overall accuracy, precision, recall, and F1-score
test_accuracy = accuracy_score(all_test_labels, all_test_preds)
test_precision, test_recall, test_f1, _ = precision_recall_fscore_support(all_test_labels, all_test_preds, average='binary', zero_division=0)

# 7. Print the calculated evaluation metrics
print("\n--- Test Set Evaluation Results ---")
print(f"Test Loss: {avg_test_loss:.4f}")
print(f"Test Accuracy: {test_accuracy:.4f}")
print(f"Test Precision: {test_precision:.4f}")
print(f"Test Recall: {test_recall:.4f}")
print(f"Test F1-score: {test_f1:.4f}")

Test Dataset and DataLoader created successfully.
Model moved to cuda and set to evaluation mode.
Starting evaluation on the test set...


Evaluating Test Set:   0%|          | 0/1578 [00:00<?, ?it/s]


--- Test Set Evaluation Results ---
Test Loss: 0.0865
Test Accuracy: 0.9674
Test Precision: 0.9780
Test Recall: 0.9688
Test F1-score: 0.9734


## References

\[1\] R. A. R. Soto, K. Koch, A. Khan, B. Chen, M. Bishop, and N. Andrews, “Few-Shot Detection of Machine-Generated Text using Style Representations,” Lawrence Livermore National Laboratory/Johns Hopkins University Technical Report, 2024.

\[2\] R. A. R. Soto, B. Chen, and N. Andrews, “Language Models Optimized to Fool Detectors Still Have a Distinct Style (And How to Change It),” arXiv e-print arXiv:2505.14608, May 2025.